### Example

Here we define some graph that would benefit from L1 optimization by choosing good order of nodes.

In [ ]:
import networkx as nx  # pip install networkx
from pgd_utils import PSDecomposer  # see ReadMe.md


# Let us generate a simple initiator graph g_prime
# 0 --- 1 --- 5 --- 6
# |   /   \       / |
# | /       \   /   |
# 3 --- 2 --- 4 --- 7

g_prime  = nx.Graph()
g_prime.add_node(0)
g_prime.add_node(1)
g_prime.add_node(2)
g_prime.add_node(3)
g_prime.add_node(4)
g_prime.add_node(5)
g_prime.add_node(6)
g_prime.add_node(7)
# self loops to make kronecker graph 100% connected
g_prime.add_edge(0, 0)
g_prime.add_edge(1, 1)
g_prime.add_edge(2, 2)
g_prime.add_edge(3, 3)
g_prime.add_edge(4, 4)
g_prime.add_edge(5, 5)
g_prime.add_edge(6, 6)
g_prime.add_edge(7, 7)
g_prime.add_edge(0, 1)
g_prime.add_edge(3, 0)
g_prime.add_edge(1, 3)
g_prime.add_edge(3, 2)
g_prime.add_edge(4, 2)
g_prime.add_edge(1, 4)
g_prime.add_edge(1, 5)
g_prime.add_edge(5, 6)
g_prime.add_edge(6, 4)
g_prime.add_edge(4, 7)
g_prime.add_edge(7, 6)
nx.draw(g_prime)

Let us decompose the adjacency matrix of the initiator graph into Pauli strings

In [ ]:
from functools import reduce

# These two functions are to get sorted amplitudes of Pauli strings (descending order)

def get_amplitudes(decomposition):
    for (_, ampls) in decomposition:
        yield from map(abs, ampls)

def get_sorted_amplitudes(decomposition):
    return sorted(get_amplitudes(decomposition), reverse=True)

# This function returns l1 error vs number of pauli strings included

def l1_relative_error(decomposition):
    l1 = sum(get_amplitudes(decomposition))
    l1_err = reduce(lambda acc, val: acc + [acc[-1] - val / l1] if acc else [1 - val / l1], get_sorted_amplitudes(ps_decomposition), None)
    return l1_err

# This function comutes number of non-zero Pauli strings in the decomposition

def get_number(decomposition):
    return sum(map(lambda x: len(x[1]), decomposition))

# Here we run decomposition

ps_decomposer = PSDecomposer()
for (id1, id2) in g_prime.edges:
    ps_decomposer.add_edge(id1, id2, 1.)
ps_decomposition = ps_decomposer.decompose()
for (order, (pstrs, weights)) in enumerate(ps_decomposition):
    print(f"For order {order} one has pauli strings {pstrs} with weights {weights}")
print("L1 norm in the Pauli basis: ", sum(get_amplitudes(ps_decomposition)))
print("Number of Pauli strings is ", get_number(ps_decomposition))
print("Total number of Pauli strings necessary to decompose any graph of given size is 64")

Now let us do the same, but with Pauli basis L1 norm minimization wrt nodes ordering in the graph

In [ ]:
optimal_order = ps_decomposer.pauli_optimize()
ps_decomposition_optimized = ps_decomposer.decompose()
for (order, (pstrs, weights)) in enumerate(ps_decomposition_optimized):
    print(f"For order {order} one has pauli strings {pstrs} with weights {weights}")
print("L1 norm in the Pauli basis: ", sum(get_amplitudes(ps_decomposition_optimized)))
print("Optimal order of nodes is ", optimal_order)
print("Number of Pauli strings is ", get_number(ps_decomposition_optimized))
print("Total number of Pauli strings necessary to decompose any graph of given size is 64")

One can note that the L1 norm and number of Pauli strings is smaller, what is expected

We know that for a kronecker product graph amplitudes of Pauli strings are given by tensor product of amplitudes of an initiator graph. Let us consider a Kronecker graph built by 4 initiator graphs. For this kronecker graph let us compare amplitudes of decomposition with fixed order of nodes with amplitudes of decomposition with optimized order of nodes.

In [ ]:
import matplotlib.pyplot as plt
from numpy import array, kron, sort, cumsum

amplitudes = array(list(get_amplitudes(ps_decomposition)))
optimized_amplitudes = array(list(get_amplitudes(ps_decomposition_optimized)))
kronecker_amplitudes = sort(kron(amplitudes, kron(amplitudes, kron(amplitudes, amplitudes))))
optimized_kronecker_amplitudes = sort(kron(optimized_amplitudes, kron(optimized_amplitudes, kron(optimized_amplitudes, optimized_amplitudes))))
relative_l1_err = (1 - cumsum(kronecker_amplitudes) / sum(kronecker_amplitudes))
optimized_relative_l1_err = (1 - cumsum(optimized_kronecker_amplitudes) / sum(optimized_kronecker_amplitudes))
plt.title("Pauli basis L1 error VS number of Pauli strings in the decomposition")
plt.plot(relative_l1_err, label="Decomposition with fixed order of nodes")
plt.plot(optimized_relative_l1_err, label="Decomposition with optimized order of nodes")
plt.vlines((8 ** 2) ** 4, ymax=1, ymin=1e-3, colors="red", label="Number of Pauli strings to decompose any graph")
plt.ylabel("L1 relative error")
plt.xlabel("Number of Paulis trings included")
plt.xscale("log")
plt.yscale("log")
plt.ylim(top=1, bottom=1e-3)
plt.legend()